# 00 - Initial Configuration

## 🎯 Objetivo
Crear, compilar y deployar el pipeline RAG en OpenShift AI Data Science Pipelines.
Este notebook toma los components desarrollados en el notebook anterior y los prepara para deployment real.

## 📋 Lo que Haremos
1. **Configurar MinIO con buckets necesarios**

## 🔧 Setup Inicial

In [ ]:
import os

import yaml
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Verificar KFP y dependencias
try:
    import kfp
    from kfp import dsl, compiler
    from kfp.client import Client
    from kfp.dsl import component, pipeline, Input, Output, Dataset
    print(f"✅ KubeFlow Pipelines disponible: {kfp.__version__}")
except ImportError:
    print("❌ Instalando KubeFlow Pipelines...")
    !pip install kfp>=2.0.0

# Verificar MinIO client
try:
    from minio import Minio
    print("✅ MinIO client disponible")
except ImportError:
    print("❌ Instalando MinIO client...")
    !pip install minio

print("🚀 Notebook de deployment iniciado")
print(f"📁 Directorio actual: {os.getcwd()}")

✅ KubeFlow Pipelines disponible: 2.12.1
✅ MinIO client disponible
🚀 Notebook de deployment iniciado
📁 Directorio actual: /opt/app-root/src


## ⚙️ Configuration Files

Creamos los archivos de configuración necesarios para el deployment

In [1]:
def create_configuration_files():
    """Crear archivos de configuración del proyecto"""
    
    # 1. Pipeline Configuration
    pipeline_config = {
        "pipeline": {
            "name": "rag-document-processing-v1",
            "version": "1.0.0",
            "description": "RAG Document Processing Pipeline para OpenShift AI",
            "author": "Carlos Estay",
            "github": "pkstaz",
            "created": datetime.now().isoformat()
        },
        "components": {
            "base_image": "pytorch/pytorch:2.0.1-cuda11.7-cudnn8-devel",
            "extract_text": {
                "cpu_limit": "500m",
                "memory_limit": "1Gi",
                "retry_limit": 3
            },
            "chunk_text": {
                "cpu_limit": "500m", 
                "memory_limit": "1Gi",
                "retry_limit": 3
            },
            "generate_embeddings": {
                "cpu_limit": "1000m",
                "memory_limit": "4Gi", 
                "retry_limit": 2,
                "gpu_limit": "0"  # Set to "1" si hay GPU disponible
            },
            "index_elasticsearch": {
                "cpu_limit": "500m",
                "memory_limit": "2Gi",
                "retry_limit": 3
            }
        },
        "default_parameters": {
            "chunk_size": 512,
            "chunk_overlap": 50,
            "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
            "es_index": "rag-documents"
        },
        "storage": {
            "minio": {
                "endpoint": "minio:9000",
                "access_key": "minio",
                "secret_key": "minio123",
                "bucket_raw": "raw-documents",
                "bucket_processed": "processed-documents",
                "bucket_failed": "failed-documents",
                "bucket_pipeline": "pipeline"
            },
            "elasticsearch": {
                "endpoint": "elasticsearch:9200",
                "index_prefix": "rag-",
                "replicas": 0,
                "shards": 1
            }
        }
    }
    
    with open('config/pipeline_config.yaml', 'w') as f:
        yaml.dump(pipeline_config, f, indent=2, default_flow_style=False)
    
    print("✅ Creado: config/pipeline_config.yaml")
    
    # 2. Secrets Template
    secrets_template = {
        "apiVersion": "v1",
        "kind": "Secret",
        "metadata": {
            "name": "rag-pipeline-secrets",
            "namespace": "rag-openshift-ai",
            "labels": {
                "app": "rag-pipeline",
                "author": "carlos-estay",
                "github": "pkstaz"
            }
        },
        "type": "Opaque",
        "stringData": {
            "minio-access-key": "minio",
            "minio-secret-key": "minio123",
            "elasticsearch-username": "",
            "elasticsearch-password": "",
            "pipeline-webhook-token": "rag-pipeline-token-change-me"
        }
    }
    
    with open('config/secrets.yaml', 'w') as f:
        yaml.dump(secrets_template, f, indent=2, default_flow_style=False)
    
    print("✅ Creado: config/secrets.yaml")
    
    # 3. Requirements file
    requirements_content = '''# RAG Pipeline Requirements
# Author: Carlos Estay (github: pkstaz)
# Core KFP
kfp>=2.0.0

# Document Processing
PyPDF2==3.0.1
python-docx==0.8.11
chardet==5.2.0

# Text Processing
tiktoken==0.5.1
langchain==0.0.350

# ML/Embeddings
sentence-transformers==2.2.2
torch>=2.0.1
numpy==1.24.3

# Storage/DB
minio==7.1.17
elasticsearch==8.11.0

# Web/API (for webhook)
flask==2.3.3
requests==2.31.0

# Utilities
pyyaml==6.0.1
python-dateutil==2.8.2
'''
    
    with open('requirements.txt', 'w') as f:
        f.write(requirements_content)
    
    print("✅ Creado: requirements.txt")
    
    # 4. Environment variables template
    env_template = '''# RAG Pipeline Environment Variables
# Author: Carlos Estay (github: pkstaz)
# Copy to .env and modify as needed

# MinIO Configuration
MINIO_ENDPOINT=minio:9000
MINIO_ACCESS_KEY=minio
MINIO_SECRET_KEY=minio123
MINIO_SECURE=false

# ElasticSearch Configuration  
ES_ENDPOINT=elasticsearch:9200
ES_INDEX=rag-documents
ES_USERNAME=
ES_PASSWORD=

# Pipeline Configuration
CHUNK_SIZE=512
CHUNK_OVERLAP=50
EMBEDDING_MODEL=sentence-transformers/all-MiniLM-L6-v2

# OpenShift AI Pipeline
KFP_HOST=http://ml-pipeline:8888
PIPELINE_NAMESPACE=rag-openshift-ai

# Webhook Configuration
WEBHOOK_PORT=8080
WEBHOOK_TOKEN=rag-pipeline-token-change-me
'''
    
    with open('config/env.template', 'w') as f:
        f.write(env_template)
    
    print("✅ Creado: config/env.template")
    
    print("\n📋 Archivos de configuración creados:")
    print("  config/pipeline_config.yaml - Configuración del pipeline")
    print("  config/secrets.yaml - Template de secrets K8s")
    print("  requirements.txt - Dependencias Python")  
    print("  config/env.template - Variables de ambiente")

# Crear archivos de configuración
create_configuration_files()

NameError: name 'datetime' is not defined

## 🪣 Configuración de MinIO - Crear Buckets

Configuramos MinIO y creamos todos los buckets necesarios incluyendo el bucket "pipeline"

In [32]:
def setup_minio_buckets_complete():
    """Configurar MinIO y crear todos los buckets necesarios incluyendo 'pipeline'"""
    
    print("🔧 Configurando MinIO y creando buckets completos...")
    
    # Configuración MinIO
    MINIO_ENDPOINT = "minio:9000"
    MINIO_ACCESS_KEY = "minio"
    MINIO_SECRET_KEY = "minio123"
    
    try:
        from minio import Minio
        from minio.error import S3Error
        
        # Conectar a MinIO
        print(f"🔗 Conectando a MinIO: {MINIO_ENDPOINT}")
        
        minio_client = Minio(
            MINIO_ENDPOINT,
            access_key=MINIO_ACCESS_KEY,
            secret_key=MINIO_SECRET_KEY,
            secure=False  # HTTP para desarrollo
        )
        
        # Verificar conectividad
        try:
            buckets = minio_client.list_buckets()
            print(f"✅ Conectado a MinIO exitosamente")
            print(f"📊 Buckets existentes: {len(buckets)}")
            for bucket in buckets:
                print(f"  - {bucket.name} (creado: {bucket.creation_date})")
        except Exception as e:
            print(f"❌ Error conectando a MinIO: {str(e)}")
            return False
        
        # Buckets necesarios para RAG pipeline + OpenShift AI DSP
        required_buckets = [
            {
                "name": "pipeline",
                "description": "Bucket requerido por OpenShift AI Data Science Pipelines"
            },
            {
                "name": "raw-documents",
                "description": "Documentos originales subidos por usuarios"
            },
            {
                "name": "processed-documents", 
                "description": "Documentos procesados exitosamente"
            },
            {
                "name": "failed-documents",
                "description": "Documentos que fallaron en procesamiento"
            },
            {
                "name": "test-datasets",
                "description": "Datasets de prueba para desarrollo"
            }
        ]
        
        print(f"\n🪣 Creando buckets necesarios...")
        
        created_buckets = []
        existing_buckets = []
        
        for bucket_info in required_buckets:
            bucket_name = bucket_info["name"]
            description = bucket_info["description"]
            
            try:
                # Verificar si bucket ya existe
                if minio_client.bucket_exists(bucket_name):
                    existing_buckets.append(bucket_name)
                    print(f"  ℹ️ Bucket ya existe: {bucket_name}")
                else:
                    # Crear bucket
                    minio_client.make_bucket(bucket_name)
                    created_buckets.append(bucket_name)
                    print(f"  ✅ Bucket creado: {bucket_name}")
                
                print(f"    📝 {description}")
                    
            except S3Error as e:
                print(f"  ❌ Error con bucket {bucket_name}: {str(e)}")
        
        print(f"\n📊 Resumen de buckets:")
        print(f"  ✅ Creados: {len(created_buckets)}")
        print(f"  ℹ️ Ya existían: {len(existing_buckets)}")
        
        # Listar todos los buckets después de la configuración
        final_buckets = minio_client.list_buckets()
        print(f"\n🗂️ Buckets disponibles:")
        for bucket in final_buckets:
            print(f"  - {bucket.name}")
        
        # Verificar bucket crítico para OpenShift AI
        if minio_client.bucket_exists("pipeline"):
            print(f"\n✅ Bucket 'pipeline' configurado correctamente para OpenShift AI DSP")
        else:
            print(f"\n❌ Bucket 'pipeline' no se pudo crear - requerido para OpenShift AI DSP")
            return False
        
        return True
        
    except ImportError:
        print("❌ MinIO client no disponible. Instalar con: pip install minio")
        return False
    except Exception as e:
        print(f"❌ Error configurando MinIO: {str(e)}")
        return False

def upload_test_document():
    """Subir documento de prueba al bucket raw-documents"""
    
    print(f"\n📄 Subiendo documento de prueba...")
    
    try:
        from minio import Minio
        import tempfile
        
        # Conectar a MinIO
        minio_client = Minio(
            "minio:9000",
            access_key="minio",
            secret_key="minio123",
            secure=False
        )
        
        # Crear documento de prueba más detallado
        test_content = """# Documento de Prueba RAG Pipeline - OpenShift AI
        
Este es un documento de prueba para validar el pipeline RAG en OpenShift AI Data Science Pipelines.

## Configuración del Ambiente

### MinIO Buckets Configurados:
- **pipeline**: Bucket requerido por OpenShift AI DSP
- **raw-documents**: Documentos originales 
- **processed-documents**: Documentos procesados exitosamente
- **failed-documents**: Documentos con errores de procesamiento
- **test-datasets**: Datasets para desarrollo y pruebas

### Pipeline de Procesamiento RAG

El sistema procesa documentos siguiendo estos pasos:

1. **Extracción de texto**: 
   - Soporta PDF, DOCX, TXT
   - Detecta encoding automáticamente
   - Extrae metadatos del documento

2. **Chunking inteligente**:
   - Chunks de 512 tokens con 50 de overlap
   - Usa tiktoken para conteo preciso
   - Mantiene contexto entre fragmentos

3. **Generación de embeddings**:
   - Modelo: sentence-transformers/all-MiniLM-L6-v2
   - Vectores de 384 dimensiones
   - Normalización para cosine similarity

4. **Indexación en ElasticSearch**:
   - Índice híbrido (texto + vectores)
   - Búsqueda semántica y por palabras clave
   - Metadatos preservados para trazabilidad

## Información del Proyecto

- **Autor**: Carlos Estay
- **GitHub**: pkstaz  
- **Pipeline**: rag-document-processing-v1
- **Platform**: OpenShift AI Data Science Pipelines

Este documento será procesado automáticamente cuando el webhook detecte 
su upload al bucket raw-documents.

## Testing del Pipeline

Para validar el funcionamiento:

1. Este documento se procesa automáticamente
2. Se generan aproximadamente 4-6 chunks
3. Cada chunk obtiene un embedding de 384 dimensiones  
4. Se indexa en ElasticSearch con ID único
5. Está disponible para búsqueda semántica

¡Pipeline RAG funcionando en OpenShift AI!
"""
        
        # Crear archivo temporal
        with tempfile.NamedTemporaryFile(mode='w', suffix='.txt', delete=False, encoding='utf-8') as f:
            f.write(test_content)
            temp_file_path = f.name
        
        # Subir archivo a MinIO
        object_name = "test-document-openshift-ai.txt"
        bucket_name = "raw-documents"
        
        try:
            minio_client.fput_object(
                bucket_name=bucket_name,
                object_name=object_name,
                file_path=temp_file_path,
                content_type="text/plain"
            )
            
            print(f"✅ Documento subido: {bucket_name}/{object_name}")
            print(f"📏 Tamaño: {len(test_content)} caracteres")
            print(f"📊 Palabras aproximadas: {len(test_content.split())}")
            
            # Verificar que se subió correctamente
            try:
                stat = minio_client.stat_object(bucket_name, object_name)
                print(f"📋 Verificación:")
                print(f"  Size: {stat.size} bytes")
                print(f"  Content-Type: {stat.content_type}")
                print(f"  Last-Modified: {stat.last_modified}")
            except Exception as e:
                print(f"⚠️ No se pudo verificar el archivo: {str(e)}")
            
            return {
                "bucket": bucket_name,
                "object": object_name,
                "size": len(test_content),
                "word_count": len(test_content.split()),
                "success": True
            }
            
        except Exception as e:
            print(f"❌ Error subiendo archivo: {str(e)}")
            return None
            
        finally:
            # Limpiar archivo temporal
            import os
            if os.path.exists(temp_file_path):
                os.unlink(temp_file_path)
        
    except Exception as e:
        print(f"❌ Error en upload de documento de prueba: {str(e)}")
        return None

# Ejecutar configuración completa de MinIO
print("🚀 Configuración completa de MinIO para OpenShift AI DSP")
print("=" * 60)

minio_setup_success = setup_minio_buckets_complete()

# Subir documento de prueba si MinIO está configurado
if minio_setup_success:
    test_upload_result = upload_test_document()
    
    if test_upload_result:
        print(f"\n🎯 MINIO CONFIGURADO PARA OPENSHIFT AI DSP:")
        print(f"  ✅ Bucket 'pipeline' creado (requerido por OpenShift AI)")
        print(f"  ✅ Buckets RAG creados (raw-documents, processed-documents, etc.)")
        print(f"  ✅ Documento de prueba subido: {test_upload_result['object']}")
        print(f"  📊 Tamaño: {test_upload_result['word_count']} palabras")
        print(f"\n🚀 LISTO PARA CONFIGURAR OPENSHIFT AI DSP")
    else:
        print(f"\n⚠️ MinIO configurado pero falta documento de prueba")
else:
    print(f"\n❌ Configurar MinIO antes de continuar")

🚀 Configuración completa de MinIO para OpenShift AI DSP
🔧 Configurando MinIO y creando buckets completos...
🔗 Conectando a MinIO: minio:9000
✅ Conectado a MinIO exitosamente
📊 Buckets existentes: 6
  - failed-documents (creado: 2025-07-01 17:40:26.746000+00:00)
  - pipeline (creado: 2025-07-01 17:46:35.035000+00:00)
  - processed-documents (creado: 2025-07-01 17:40:26.739000+00:00)
  - rag-documents (creado: 2025-07-01 16:45:51.180000+00:00)
  - raw-documents (creado: 2025-07-01 17:40:26.732000+00:00)
  - test-datasets (creado: 2025-07-01 17:40:26.753000+00:00)

🪣 Creando buckets necesarios...
  ℹ️ Bucket ya existe: pipeline
    📝 Bucket requerido por OpenShift AI Data Science Pipelines
  ℹ️ Bucket ya existe: raw-documents
    📝 Documentos originales subidos por usuarios
  ℹ️ Bucket ya existe: processed-documents
    📝 Documentos procesados exitosamente
  ℹ️ Bucket ya existe: failed-documents
    📝 Documentos que fallaron en procesamiento
  ℹ️ Bucket ya existe: test-datasets
    📝 Data